# IMDB Analysis

Data source:

https://datasets.imdbws.com/

Initialize Spark

In [1]:
import findspark
import pyspark
findspark.find()

'/Users/nics/Dev/spark-3.2.1-bin-hadoop3.2'

Set up Spark Application

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[8]").appName("IMDB").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/02 22:58:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


See if is it's up and get the UI

In [3]:
spark

We need a schema so let's import 

In [4]:
from pyspark.sql.types import *

And define schema

In [5]:
schema = StructType() \
      .add("tconst",StringType(),True) \
      .add("titleType",StringType(),True) \
      .add("primaryTitle",StringType(),True) \
      .add("originalTitle",StringType(),True) \
      .add("isAdult",IntegerType(),True) \
      .add("startYear",IntegerType(),True) \
      .add("endYear",IntegerType(),True) \
      .add("runtimeMinutes",DoubleType(),True) \
      .add("genres",StringType(),True) 

That's quite important, without it's a complete mess

Credit

https://medium.com/@smdbilal.vt5815/csv-bad-record-handling-and-its-complications-pyspark-f3b871d652ba

In [6]:
spark.conf.set("spark.sql.csv.parser.columnPruning.enabled",False)

Now it's time to load

In [7]:
file = "/Users/nics/Downloads/title.basics.tsv.gz"  # Should be some file on your system
dataset = spark.read\
.option("mode","DROPMALFORMED")\
.option("Header",True)\
.option("delimiter","\t")\
.schema(schema)\
.csv(file)

dataset


DataFrame[tconst: string, titleType: string, primaryTitle: string, originalTitle: string, isAdult: int, startYear: int, endYear: int, runtimeMinutes: double, genres: string]

In [8]:
dataset.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: integer (nullable = true)
 |-- startYear: integer (nullable = true)
 |-- endYear: integer (nullable = true)
 |-- runtimeMinutes: double (nullable = true)
 |-- genres: string (nullable = true)



In [9]:
dataset.describe('isAdult').show()

+-------+--------------------+
|summary|             isAdult|
+-------+--------------------+
|  count|             8890187|
|   mean|0.031045916132022868|
| stddev|  0.1734418363812059|
|    min|                   0|
|    max|                   1|
+-------+--------------------+



In [10]:
dataset.show(10)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0025509| tvSeries|      Les Misérables|      Les misérables|      0|     1934|   1934|         279.0|               Drama|
|tt0035599| tvSeries|Voice of Fireston...|Voice of Fireston...|      0|     1943|   1947|          15.0|                  \N|
|tt0035803| tvSeries|The German Weekly...|Die Deutsche Woch...|      0|     1940|   1945|          12.0|    Documentary,News|
|tt0038276| tvSeries|   You Are an Artist|   You Are an Artist|      0|     1946|   1955|          15.0|           Talk-Show|
|tt0039120| tvSeries|           Americana|           Americana|      0|     1947|   1949|          30.0|    Family,Gam

In [11]:
file2 = "/Users/nics/Downloads/title.ratings.tsv.gz"  # Should be some file on your system
ratings = spark.read.option("Header",True).option("delimiter","\t").csv(file2)
ratings

DataFrame[tconst: string, averageRating: string, numVotes: string]

In [12]:
ratings.show()

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    1878|
|tt0000002|          5.9|     248|
|tt0000003|          6.5|    1650|
|tt0000004|          5.8|     160|
|tt0000005|          6.2|    2475|
|tt0000006|          5.2|     165|
|tt0000007|          5.4|     771|
|tt0000008|          5.4|    2017|
|tt0000009|          5.3|     193|
|tt0000010|          6.9|    6778|
|tt0000011|          5.3|     344|
|tt0000012|          7.4|   11638|
|tt0000013|          5.7|    1797|
|tt0000014|          7.1|    5218|
|tt0000015|          6.2|     992|
|tt0000016|          5.9|    1406|
|tt0000017|          4.6|     305|
|tt0000018|          5.3|     561|
|tt0000019|          5.2|      30|
|tt0000020|          4.8|     332|
+---------+-------------+--------+
only showing top 20 rows



In [13]:
joined = dataset.join(ratings,dataset.tconst == ratings.tconst)

In [14]:
joined

DataFrame[tconst: string, titleType: string, primaryTitle: string, originalTitle: string, isAdult: int, startYear: int, endYear: int, runtimeMinutes: double, genres: string, tconst: string, averageRating: string, numVotes: string]

In [15]:
joined.head(10)

[Row(tconst='tt0025509', titleType='tvSeries', primaryTitle='Les Misérables', originalTitle='Les misérables', isAdult=0, startYear=1934, endYear=1934, runtimeMinutes=279.0, genres='Drama', tconst='tt0025509', averageRating='8.3', numVotes='1781'),
 Row(tconst='tt0035803', titleType='tvSeries', primaryTitle='The German Weekly Review', originalTitle='Die Deutsche Wochenschau', isAdult=0, startYear=1940, endYear=1945, runtimeMinutes=12.0, genres='Documentary,News', tconst='tt0035803', averageRating='7.9', numVotes='51'),
 Row(tconst='tt0039120', titleType='tvSeries', primaryTitle='Americana', originalTitle='Americana', isAdult=0, startYear=1947, endYear=1949, runtimeMinutes=30.0, genres='Family,Game-Show', tconst='tt0039120', averageRating='3.0', numVotes='14'),
 Row(tconst='tt0039123', titleType='tvSeries', primaryTitle='Kraft Theatre', originalTitle='Kraft Television Theatre', isAdult=0, startYear=1947, endYear=1958, runtimeMinutes=60.0, genres='Drama', tconst='tt0039123', averageRating

In [16]:
joined.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: integer (nullable = true)
 |-- startYear: integer (nullable = true)
 |-- endYear: integer (nullable = true)
 |-- runtimeMinutes: double (nullable = true)
 |-- genres: string (nullable = true)
 |-- tconst: string (nullable = true)
 |-- averageRating: string (nullable = true)
 |-- numVotes: string (nullable = true)



In [17]:
sorted(joined.groupBy('isAdult').agg({'numVotes': 'sum'}).collect())

[Row(isAdult=0, sum(numVotes)=1237338202.0),
 Row(isAdult=1, sum(numVotes)=853190.0)]